<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Parking_spot_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Data](https://drive.google.com/file/d/11DyZ165lZGzULEZSQNofyy9A8xaYgFJ2)

In [7]:
import os
import shutil
import skimage
import sklearn
import numpy as np
import pandas as pd

In [6]:
zip_folder = '/content/drive/MyDrive/DATA/car-parking-slot-empty-notempty.zip'

In [18]:
shutil.copyfile(zip_folder, '/content/data.zip')

'/content/data.zip'

In [21]:
!unzip '/content/data.zip'
!rm '/content/data.zip'